# Introduction

WaterTAP is built on IDAES [LINK] and Pyomo [LINK]. This tutorial intends to introduce some important concepts from these two packages that are used extensively in WaterTAP.

Pyomo is software package for formulating, solving, and analyzing optimization models.

The fundamental Pyomo components are:
- `Var`: unknown values in the model, i.e., values we are trying to optimize in our model.
    - e.g., concentration of a solute under given equilibrium conditions, operating pressure for a reverse osmosis process
- `Param`: data that must be provided in order to find optimal values for the decision variables. 
    - e.g., the molecular weights of solutes, the cost of an RO membrane
- `Constraint`: equations relating the `Var`s and `Param`s to one another.
- `Objective`: equiation that we are trying to minimize (or maximize) in our model

When declaring `Var` components, there are some important arguments to consider. Note that these are *optional* arguments, but proper assignment will make for a better formulated model:
- `initialize`: a value or function used to initialize the variable in the model. Depending on how complicated the model is and the formulation of the model, initialization values can impact whether or not the model solves. At the very least, you should provide this keyword with an educated guess of the optimized value.
- `bounds`: the numerical bounds on the `Var`.
- `domain`: a super-set of the `bounds` on the `Var` but can be provided without `bounds`
- `units`: the units for the `Var`
- `doc`: string description of `Var`

Many of these are available on `Param`s as well.






# Make Necessary Imports

I have imported only those necessary for this workshop.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyomo.environ import (
    ConcreteModel,
    Objective,
    Var,
    Set,
    Param,
    Constraint,
    check_optimal_termination,
    assert_optimal_termination,
    value,
    NonNegativeReals,
    SolverFactory,
    exp,
    log,
    units as pyunits,
)
from pyomo.util.check_units import assert_units_consistent
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.core.solvers import get_solver

# Building Pyomo Model

This is where you create variables, constraints, etc. prior to solving your model.

The `ConcreteModel` is the base object for a Pyomo model. All other variables, parameters, constraints, etc. are built upon this object.
    
No arguments are necessary. Convention is to call the local object `m` or `model`, but it can be called whatever you want.


In [ ]:
# Instantiate model as instance of a ConcreteModel object
m = ConcreteModel()

## We call the `.display()` method to see variables, objectives, and constraints on the model

In [ ]:
m.display()

## There are no variables or constraints in our model because we haven't added any.

### Let's add three variables - `A`, `B`, and `C`

In [ ]:
m.A = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable A for one leg of a right triangle",
)
m.B = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable B for the other leg of a right triangle",
)
m.C = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable C for the hypotenuse of a right triangle",
)

m.display()

## Now we add constraints relating our three variables:

### $A^2 + B^2 = C^2$

In [ ]:
m.pythagorean = Constraint(expr=m.A**2 + m.B**2 == m.C**2, doc="Pythagorean theorem")

# Degrees of freedom (DOF)
Ok, we have variables and an equation relating them.

We can view the degrees of freedom for the model using the `degrees_of_freedom()` function

In [ ]:
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

## With >0 DOF, we can still solve...

However, with >0 DOF, the resulting values will be close to our initialization values of 10. 

Let's add more `Constraint` to our model and a `Param` - an arbitrary relation between A and B:

## $ A = 3 B $


In [ ]:
m.P = Param(
    initialize=3,
    mutable=True,
    units=pyunits.dimensionless,
    doc="Ratio between two legs of traiangle",
)


# m.A_B_relation = Constraint(expr=m.A == m.B * m.P, doc="Equation relating A and B")
@m.Constraint(doc="Equation relating A and B")
def A_B_relation(m):
    return m.A == m.B * m.P

## Now we create our solver object with the WaterTAP solver 

### Alternatively, you can use the version of IPOPT from `SolverFactory()`

In [ ]:
# solver = SolverFactory("ipopt")
solver = get_solver()

# And recheck our DOF
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
print(results)

## There is still 1 DOF, so we fix one of our variables using the `.fix()` method.

In [ ]:
m.A.fix(5)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

## We can just as easily `.unfix()` the variable.

In [ ]:
m.A.unfix()

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

## We will re-fix and then solve with 0 DOF.

In [ ]:
m.A.fix(5)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)

m.display()

## Note that for `Param` components, we can also change the value using the `set_value()` method

The `.fix()` method will yield an error.

In [ ]:
m.P.set_value(23)
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)

m.display()

## We want to check the termination condition of the solver
The local variable `results` is where the solver status is stored.

We can also test optimal solve with `check_optimal_termination(results)` or `assert_optimal_termination(results)`

You can also just `print(results)`.

In [ ]:
print(f"Termination Condition {results.solver.termination_condition}\n")

print(check_optimal_termination(results))
assert_optimal_termination(results)

print(results)

## Now let's see an error

Fixing `B` to -45 is outside the `domain` of the variable - this will produce an error.

Similarly, this will result in an overspecified problem (-1 degrees of freedom), which will also cause an error.

In [ ]:
m.B.fix(-45)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# results = solver.solve(m)

## We can remedy this error

### First, we can re-define the lower bound on `B` 

In [ ]:
m.B.setlb(-50)
m.B.display()
print(f"Degrees of Freedom: {degrees_of_freedom(m)}")

### However, Pyomo will ignore this adjustment because the `domain` is `NonNegativeReals`

In [ ]:
from pyomo.environ import Reals

m.B.domain = Reals
m.B.setlb(-50)
m.B.display()


### We can also deactivate a constraint to return to 0 DOF

In [ ]:
m.A_B_relation.deactivate()

print(f"Degrees of Freedom: {degrees_of_freedom(m)}")

results = solver.solve(m)
assert_optimal_termination(results)
m.display()

### Return to the original formulation of the model

In [ ]:
# Unfix B, set P back to 3, and reactivate the constraint
m.B.unfix()
m.P.set_value(3)
m.A_B_relation.activate()

results = solver.solve(m)
print(f"Model solve = {results.solver.termination_condition}\n")
m.display()

## You can access the value of a Var or Param by using the `value()` function or "calling" it - e.g., `m.A()`

In [ ]:
# e.g., value(m.A)
print(f"A = {value(m.A)}", f"\nB = {value(m.B)}", f"\nC = {value(m.C)}")
# e.g., m.A()
print(f"\nA = {m.A()}", f"\nB = {m.B()}", f"\nC = {m.C()}")

## Put it all in a `build` function

In [ ]:
# Putting it all together


def build_pythagorean():
    m = ConcreteModel()
    m.A = Var(
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        units=pyunits.m,
        doc="Model variable A for one leg of a right triangle",
    )
    m.B = Var(
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        units=pyunits.m,
        doc="Model variable B for the other leg of a right triangle",
    )
    m.C = Var(
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        units=pyunits.m,
        doc="Model variable C for the hypotenuse of a right triangle",
    )
    m.P = Param(
        initialize=3,
        mutable=True,
        units=pyunits.dimensionless,
        doc="Ratio between A and B",
    )

    m.A_B_relation = Constraint(expr=m.A == m.B * m.P)

    m.pythagorean = Constraint(
        expr=m.A**2 + m.B**2 == m.C**2, doc="Pythagorean theorem"
    )

    return m


m = build_pythagorean()
m.A.fix(5)

results = solver.solve(m)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

m.display()

print(f"\nA = {m.A()}")
print(f"B = {m.B()}")
print(f"C = {m.C()}")

## Using `if` statements (example of what you can't do)

When you are building your model (prior to solving it), your `Var`s don't have optimized values. They have initialized values. So constructing a `Constraint` with an initialized value doesn't make any sense. Fortunately, Pyomo prevents you from doing this.

In [ ]:
m = build_pythagorean()
m.A.fix(5)

# if m.B() > 3:
if m.B > 3:
    m.A_B_relation = Constraint(expr=m.A == m.B * m.P - 2)
else:
    m.A_B_relation = Constraint(expr=m.A == m.B * m.P)

# Indexing variables, constraints, and parameters

In the context of water treatment, you might want to do this if you are tracking multiple constituents through the treatment process and have to perform, for example, the same equilibrium calculation (but with different data) for each of them.

Any model `Component` in Pyomo can be indexed with any number of indexes. For example, if your model is dynamic (non-steady state) and you are tracking multiple constituents, you could index your `Var`s and `Constraint`s to both time and your constituents.

When constructing your model components, the first non-keyword argument(s) are your indexes. So, if I am creating an initial concentration `Var` indexed to `ions`, it would be:


In [ ]:
m = ConcreteModel()
# Create index
ions = ["Na", "Mg", "Ca", "Sr"]

m.C_0 = Var(
    ions,
    initialize=2,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Initial ion concentration",
)


To initialize an indexed `Var`, you can pass a single value or any Python iterable (e.g. dictionary) that maps the indexes to their initial value. In the example above, I passed 2 to the initialize keyword, so the initial concentration for all my ions will be 2. If I want different initial values for each index, I could do something like:


In [ ]:
# Create index and initialization values.
ions = ["Na", "Mg", "Ca", "Sr"]
ions_init = {"Na": 1, "Mg": 2, "Ca": 3, "Sr": 4}

m.C_0 = Var(
    ions,
    initialize=ions_init,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Initial ion concentration",
)



Like `Var` objects, a `Constraint` can also be indexed by passing the index as the first argument(s). For non-indexed `Constraint`, we used the `expr=` keyword to define our relationship. With an indexed `Constraint`, we must use the `rule=` keyword and define a function to construct our `Constraint` relationship.

Let's say we can get the steady-state concentration of our ions by multiplying it by some constant parameter `K_eq` that is different for each ion. We would create a steady-state concentration `Var` (`C_ss`) and define our equilibrium calculation `Constraint` that is indexed to each ion:



In [ ]:
K_eq_init = {"Na": 1e-5, "Mg": 5.5e-6, "Ca": 3.2e-4, "Sr": 4e-4}

m.K_eq = Var(ions, initialize=K_eq_init, doc="Equilibrium constant")

m.C_ss = Var(
    ions,
    initialize=ions_init,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Steady-state ion concentration",
)


# The first argument must be the model (m), and the second must be the index
def eq_steady_state_conc(m, i):
    return m.C_ss[i] == m.C_0[i] * exp(-m.K_eq[i])


m.steady_state_conc_constr = Constraint(ions, rule=eq_steady_state_conc)



All of my model components are indexed to `ions`, so each ion has a separate equilibrium calculation in this model.

A simple example is presented below. In this case, we have two triangles we want to model: `foo` and `bar`. For the model to solve with 0 DOF, we must specify relationships for all components across all indices.


In [ ]:
def build_idx_pythagorean():
    m = ConcreteModel()
    P_data = {"cats": 3, "dogs": 8}

    m.idx = Set(initialize=["cats", "dogs"], doc="Index set for multiple triangles")

    m.A = Var(
        m.idx,
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        doc="Model variable A for one leg of a right triangle",
    )
    m.B = Var(
        m.idx,
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        doc="Model variable B for the other leg of a right triangle",
    )
    m.C = Var(
        m.idx,
        initialize=10,
        domain=NonNegativeReals,
        bounds=(0, None),
        doc="Model variable C for the hypotenuse of a right triangle",
    )
    m.P = Param(
        m.idx,
        initialize=P_data,
        mutable=True,
        units=pyunits.dimensionless,
        doc="Ratio between A and B",
    )

    def eq_A_B_relation(m, i):
        return m.A[i] == m.B[i] * m.P[i]


    m.A_B_relation = Constraint(m.idx, rule=eq_A_B_relation)


    # Rather than having a separate Constraint for each index,
    # you can pass a function as the Constraint.
    # Here I define the constraint function with the first argument as the model and the second argument as the index.
    def eq_pythagorean(m, i):
        return m.A[i] ** 2 + m.B[i] ** 2 == m.C[i] ** 2

    m.pythagorean = Constraint(m.idx, rule=eq_pythagorean)

    return m

m = build_idx_pythagorean()

m.B["cats"].fix(5)
m.B["dogs"].fix(125)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
print(f"Model solve = {results.solver.termination_condition}\n")

m.display()

# You must indicate the index when calling variables now:
for i in m.idx:
    print(f"Index = {i}")
    print(f"\tA = {m.A[i]()}")
    print(f"\tB = {m.B[i]()}")
    print(f"\tC = {m.C[i]()}")

# You can also call .display() on model Var:
m.A.display()

In [ ]:
m.P.display()

## Units (`pyunits`)

We have imported `units as pyunits`

`pyunits` are extremely useful for making easy unit conversions but also ensuring our models have consistent units.

We can define any value with any dimensions:
Let's create 100 g/L of some substance

In [ ]:
C = 100 * pyunits.g / pyunits.L

print(C)
print(value(C))
print(C())

## We can convert C to any units we want...

...as long as they are of the same kind with `pyunits.convert()`

(you can't convert g/L to acres/second)

And you can get units from any variable using `pyunits.get_units()`

In [ ]:
C2 = pyunits.convert(C, to_units=pyunits.lb / pyunits.ft**3)
C3 = pyunits.convert(C, to_units=pyunits.grain / (pyunits.acre * pyunits.foot))

print(C2(), pyunits.get_units(C2))
print(C3(), pyunits.get_units(C3))

### Units carry through operations

In [ ]:
Q = 42 * pyunits.m**3 / pyunits.hr
conc = 100 * pyunits.mg / pyunits.L

mass_flow = Q * conc

print(mass_flow(), pyunits.get_units(mass_flow))

# Or

mass_flow = pyunits.convert(Q * conc, to_units=pyunits.g / pyunits.s)

print(mass_flow(), pyunits.get_units(mass_flow))

# Example 20-5 MWR
Groundwater contains 5 g/m3 Fe and 2 g/m3 Mn

Q = 1E5 m3/d

KMnO4 is used to oxidize both

Determine:
- KMnO4 required (total and per ion)
- alkalinity consumed (total and per ion)
- quantity of sludge produced daily (total and per ion)

In [ ]:
ions = ["Fe", "Mn"]
conc_init = {"Fe": 5, "Mn": 2}
kmno4_conv_init = {"Fe": 0.94, "Mn": 1.92}
alk_conv_init = {"Fe": 1.5, "Mn": 1.21}
sludge_conv_init = {"Fe": 2.43, "Mn": 2.64}

m = ConcreteModel()

m.aq_conc = Param(
    ions,
    initialize=conc_init,
    units=pyunits.g / pyunits.m**3,
    doc="Aqueous concentration of ions",
)

m.kmno4_conversion = Param(
    ions,
    initialize=kmno4_conv_init,
    units=pyunits.g / pyunits.g,
    doc="KMnO4 required conversion",
)

m.alk_conversion = Param(
    ions, initialize=alk_conv_init, units=pyunits.g / pyunits.g, doc="Alk conversion"
)

m.sludge_conversion = Param(
    ions,
    initialize=sludge_conv_init,
    units=pyunits.g / pyunits.g,
    doc="Sludge produced conversion",
)

m.flow_in = Var(
    initialize=1000, bounds=(0, 1e6), units=pyunits.m**3 / pyunits.d, doc="Daily flow"
)

m.kmno4_required = Var(
    ions,
    bounds=(0, 1e5),
    initialize=100,
    units=pyunits.kg / pyunits.d,
    doc="Daily KMnO4 required per ion",
)

m.tot_kmno4_required = Var(
    bounds=(0, 1e5),
    initialize=100,
    units=pyunits.kg / pyunits.d,
    doc="Total KMnO4 required",
)

m.alk_consumed = Var(
    ions,
    initialize=100,
    bounds=(0, 1e5),
    units=pyunits.kg / pyunits.d,
    doc="Daily alkalinity consumed per ion",
)

m.tot_alk_consumed = Var(
    initialize=100,
    bounds=(0, 1e5),
    units=pyunits.kg / pyunits.d,
    doc="Total alkalinity consumed",
)

m.sludge_produced = Var(
    ions,
    bounds=(0, 1e5),
    initialize=100,
    units=pyunits.kg / pyunits.d,
    doc="Daily sludge production per ion",
)

m.tot_sludge_produced = Var(
    bounds=(0, 1e5),
    initialize=100,
    units=pyunits.kg / pyunits.d,
    doc="Total sludge produced",
)


def eq_kmno4_req(m, i):
    return m.kmno4_required[i] == pyunits.convert(
        m.aq_conc[i] * m.flow_in * m.kmno4_conversion[i],
        to_units=pyunits.kg / pyunits.d,
    )


m.kmno4_req_constr = Constraint(ions, rule=eq_kmno4_req)

m.tot_kmno4_req_constr = Constraint(
    expr=m.tot_kmno4_required == sum(m.kmno4_required[i] for i in ions)
)


def eq_alk_consumed(m, i):
    return m.alk_consumed[i] == pyunits.convert(
        m.aq_conc[i] * m.flow_in * m.alk_conversion[i], to_units=pyunits.kg / pyunits.d
    )


m.alk_consumed_constr = Constraint(ions, rule=eq_alk_consumed)


m.tot_alk_consumed_constr = Constraint(
    expr=m.tot_alk_consumed == sum(m.alk_consumed[i] for i in ions)
)


def eq_sludge_produced(m, i):
    return m.sludge_produced[i] == pyunits.convert(
        m.aq_conc[i] * m.flow_in * m.sludge_conversion[i],
        to_units=pyunits.kg / pyunits.d,
    )


m.sludge_produced_constr = Constraint(ions, rule=eq_sludge_produced)

m.tot_sludge_produced_constr = Constraint(
    expr=m.tot_sludge_produced == sum(m.sludge_produced[i] for i in ions)
)

m.flow_in.fix(1e5)
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
print(f"Model solve = {results.solver.termination_condition}\n")

m.display()

# Add costing
Let's say we had a cost for this process that was a function of these four things:

`C = 15200 + 320 * tot_sludge_produced ** 1.2 + 142 * tot_alk_consumed ** 0.2 + 170 * log(tot_kmno4_consumed) ** 2 + flow_in ** 0.078`

In [ ]:
m.capital_cost = Var(
    initialize=1e6,
    bounds=(0, 50e6),
    units=pyunits.dimensionless,
    doc="Unit process capital cost",
)


m.cap_constr = Constraint(
    expr=m.capital_cost
    == 15200
    + 320 * m.tot_sludge_produced**1.2
    + 142 * m.tot_alk_consumed**0.2
    + 170 * log(m.tot_kmno4_required) ** 2
    + m.flow_in**0.078
)

# You can't use standard Python math operations on Pyomo objects

Here we tried to pass a Pyomo `Var` to the Python operator `log()`. Pyomo has defined its own set of these operations for use with Pyomo objects. So if you needed to use `log()` in a `Constraint` like we have here, you must:

```python
from pyomo.environ import log
``` 

This will overwrite the Python operation (but you can still use it normally). If you are unsure if you will be using these types of operations when you build the model, you can avoid this by just importing everything from the Pyomo environment:

```python
from pyomo.environ import *
``` 

In [ ]:
from pyomo.environ import log

In [ ]:
m.cap_constr = Constraint(
    expr=m.capital_cost
    == 15200
    + 320 * m.tot_sludge_produced**1.2
    + 142 * m.tot_alk_consumed**0.2
    + 170 * log(m.tot_kmno4_required) ** 2
    + m.flow_in**0.078
)

In [ ]:
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
print(f"Model solve = {results.solver.termination_condition}\n")

m.display()

In the first version of this model, the conversion factors for KMnO4 consumption and sludge generation were `Param` - i.e., they weren't variables and couldn't be changed.

Instead, let's imagine we don't know this data *a priori* but instead have some relationships that define them. 


In [ ]:
# ions = ["Fe", "Mn"]
# conc_init = {"Fe": 5, "Mn": 2}
# kmno4_conv_init = {"Fe": 0.94, "Mn": 1.92}
# alk_conv_init = {"Fe": 1.5, "Mn": 1.21}
# sludge_conv_init = {"Fe": 2.43, "Mn": 2.64}

# m = ConcreteModel()

# m.aq_conc = Param(
#     ions,
#     initialize=conc_init,
#     units=pyunits.g / pyunits.m**3,
#     doc="Aqueous concentration of ions",
# )

# m.kmno4_conversion = Var(
#     ions,
#     bounds=(0, None),
#     initialize=kmno4_conv_init,
#     units=pyunits.g / pyunits.g,
#     doc="KMnO4 required conversion",
# )

# m.alk_conversion = Param(
#     ions, initialize=alk_conv_init, units=pyunits.g / pyunits.g, doc="Alk conversion"
# )

# m.sludge_conversion = Var(
#     ions,
#     initialize=sludge_conv_init,
#     bounds=(0, None),
#     units=pyunits.g / pyunits.g,
#     doc="Sludge produced conversion",
# )

# m.flow_in = Var(
#     initialize=1000, bounds=(0, 1e6), units=pyunits.m**3 / pyunits.d, doc="Daily flow"
# )

# m.kmno4_required = Var(
#     ions,
#     bounds=(0, 1e5),
#     initialize=100,
#     units=pyunits.kg / pyunits.d,
#     doc="Daily KMnO4 required per ion",
# )

# m.tot_kmno4_required = Var(
#     bounds=(0, 1e5),
#     initialize=100,
#     units=pyunits.kg / pyunits.d,
#     doc="Total KMnO4 required",
# )

# m.alk_consumed = Var(
#     ions,
#     initialize=100,
#     bounds=(0, 1e5),
#     units=pyunits.kg / pyunits.d,
#     doc="Daily alkalinity consumed per ion",
# )

# m.tot_alk_consumed = Var(
#     initialize=100,
#     bounds=(0, 1e5),
#     units=pyunits.kg / pyunits.d,
#     doc="Total alkalinity consumed",
# )

# m.sludge_produced = Var(
#     ions,
#     bounds=(0, 1e5),
#     initialize=100,
#     units=pyunits.kg / pyunits.d,
#     doc="Daily sludge production per ion",
# )

# m.tot_sludge_produced = Var(
#     bounds=(0, 1e5),
#     initialize=100,
#     units=pyunits.kg / pyunits.d,
#     doc="Total sludge produced",
# )

# m.capital_cost = Var(
#     initialize=1e6,
#     bounds=(0, 50e6),
#     units=pyunits.dimensionless,
#     doc="Unit process capital cost",
# )


# def eq_kmno4_sludge(m, i):
#     return (
#         m.kmno4_conversion[i] == 2.6 * m.sludge_conversion[i] - 0.1 * m.alk_consumed[i]
#     )


# m.kmno4_sludge_rel = Constraint(ions, rule=eq_kmno4_sludge)


# def eq_kmno4_req(m, i):
#     return m.kmno4_required[i] == pyunits.convert(
#         m.aq_conc[i] * m.flow_in * m.kmno4_conversion[i],
#         to_units=pyunits.kg / pyunits.d,
#     )


# m.kmno4_req_constr = Constraint(ions, rule=eq_kmno4_req)

# m.tot_kmno4_req_constr = Constraint(
#     expr=m.tot_kmno4_required == sum(m.kmno4_required[i] for i in ions)
# )


# def eq_alk_consumed(m, i):
#     return m.alk_consumed[i] == pyunits.convert(
#         m.aq_conc[i] * m.flow_in * m.alk_conversion[i], to_units=pyunits.kg / pyunits.d
#     )


# m.alk_consumed_constr = Constraint(ions, rule=eq_alk_consumed)


# m.tot_alk_consumed_constr = Constraint(
#     expr=m.tot_alk_consumed == sum(m.alk_consumed[i] for i in ions)
# )


# def eq_sludge_produced(m, i):
#     return m.sludge_produced[i] == pyunits.convert(
#         m.aq_conc[i] * m.flow_in * m.sludge_conversion[i],
#         to_units=pyunits.kg / pyunits.d,
#     )


# m.sludge_produced_constr = Constraint(ions, rule=eq_sludge_produced)

# m.tot_sludge_produced_constr = Constraint(
#     expr=m.tot_sludge_produced == sum(m.sludge_produced[i] for i in ions)
# )

# m.cap_constr = Constraint(
#     expr=m.capital_cost
#     == 15200
#     + 320 * m.tot_sludge_produced**1.2
#     + 142 * m.tot_alk_consumed**0.2
#     + 170 * log(m.tot_kmno4_required) ** 2
#     + m.flow_in**0.078
# )

# m.flow_in.fix(1e5)
# m.kmno4_conversion["Fe"].fix(1.1)
# m.kmno4_conversion["Mn"].fix(5.1)

# assert_units_consistent(m)
# dof = degrees_of_freedom(m)
# print(f"Degrees of Freedom: {dof}")

# results = solver.solve(m)
# print(f"Model solve = {results.solver.termination_condition}\n")
# assert_optimal_termination(results)
# m.display()

In [ ]:
#### ROSENBROCK EXAMPLE
# Z = (1 - X) ** 2 + 100 * (Y - X ** 2) ** 2
# Objective = minimize